In [1]:
from sentinelsat import SentinelAPI, read_geojson, geojson_to_wkt

In [2]:
from datetime import date

In [3]:
import requests

In [4]:
import threading
import multiprocessing

In [5]:
USERNAME = ["barmasushma", "barmasushma1", "SumanjaliDamarla"]
PASSWORD = ["barmasushma", "barmasushma1", "D.Sumanjali1"]
API_SOURCE = "https://scihub.copernicus.eu/dhus"
PROCESSING_LEVEL = ["Level-1", "Level-2"]
SATELLITE = "Sentinel-2"
STUDY_AREA = "StudyArea.geojson"

In [6]:
#download for different months and 2 areas, then check for processing level and cloud data

In [7]:
footprint = geojson_to_wkt(read_geojson(STUDY_AREA))

In [8]:
api = []
for i in range(len(USERNAME)):
    api.append(SentinelAPI(USERNAME[i], PASSWORD[i], API_SOURCE))

In [9]:
NOV_BEGIN = "20181101"
NOV_END = "20181130"

DEC_BEGIN = "20181201"
DEC_END = "20181231"

JAN_BEGIN = "20190101"
JAN_END = "20190131"

FEB_BEGIN = "20190201"
FEB_END = "20190228"

MAR_BEGIN = "20190301"
MAR_END = "20190331"

In [10]:
def queryForProducts(footprint, beg, end, sat):
    return api[0].query(footprint, date=(beg, end),platformname=sat)

In [11]:
def getLevelURLs(df):
    level1 = []
    for i in range(df.shape[0]):
        if PROCESSING_LEVEL[0] in df["processinglevel"][i]:
            level1.append(df["link"][i])

    level2 = []
    for i in range(df.shape[0]):
        if PROCESSING_LEVEL[1] in df["processinglevel"][i]:
            level2.append(df["link"][i])
    
    return level1, level2

In [12]:
def curateLevelProducts(products, df, level):
    keys = list(products.keys())
    for i in range(df.shape[0]):
        if level not in df["processinglevel"][i]:
            products.pop(keys[i])
    return products

In [13]:
nov_products = queryForProducts(footprint, NOV_BEGIN, NOV_END,SATELLITE)
dec_products = queryForProducts(footprint, DEC_BEGIN, DEC_END,SATELLITE)
jan_products = queryForProducts(footprint, JAN_BEGIN, JAN_END,SATELLITE)
feb_products = queryForProducts(footprint, FEB_BEGIN, FEB_END,SATELLITE)
mar_products = queryForProducts(footprint, MAR_BEGIN, MAR_END,SATELLITE)

In [14]:
nov_df = api[0].to_dataframe(nov_products)
dec_df = api[0].to_dataframe(dec_products)
jan_df = api[0].to_dataframe(jan_products)
feb_df = api[0].to_dataframe(feb_products)
mar_df = api[0].to_dataframe(mar_products)

In [15]:
nov_level1, nov_level2 = getLevelURLs(nov_df)
dec_level1, dec_level2 = getLevelURLs(dec_df)
jan_level1, jan_level2 = getLevelURLs(jan_df)
feb_level1, feb_level2 = getLevelURLs(feb_df)
mar_level1, mar_level2 = getLevelURLs(mar_df)

In [16]:
nov_data = curateLevelProducts(nov_products, nov_df, PROCESSING_LEVEL[1])
dec_data = curateLevelProducts(dec_products, dec_df, PROCESSING_LEVEL[1])
jan_data = curateLevelProducts(jan_products, jan_df, PROCESSING_LEVEL[1])
feb_data = curateLevelProducts(feb_products, feb_df, PROCESSING_LEVEL[1])
mar_data = curateLevelProducts(mar_products, mar_df, PROCESSING_LEVEL[1])

In [17]:
months = [
    "Nov", "Dec", "Jan", "Feb", "Mar"
]
levelCount = [
    [nov_df.shape[0], len(nov_level1), len(nov_level2)],
    [dec_df.shape[0], len(dec_level1), len(dec_level2)],
    [jan_df.shape[0], len(jan_level1), len(jan_level2)],
    [feb_df.shape[0], len(feb_level1), len(feb_level2)],
    [mar_df.shape[0], len(mar_level1), len(mar_level2)]
]

print("      Total  L1  L2")
for i in range(len(months)):
    print(months[i] + "   " + str(levelCount[i][0]) + "     " + str(levelCount[i][1]) + "  " + str(levelCount[i][2]))

      Total  L1  L2
Nov   40     40  0
Dec   65     42  23
Jan   90     45  45
Feb   74     37  37
Mar   84     42  42


In [ ]:
t1 = threading.Thread(api[0].download_all(dec_data), name='t1') 
t2 = threading.Thread(api[0].download_all(feb_data), name='t2')     
# starting threads 
t1.start()
threading.current_thread().name
t2.start()
threading.current_thread().name
# wait until all threads finish 
t1.join() 
t2.join() 

In [ ]:
p1 = multiprocessing.Process(target=api[0].download_all, args=(dec_data,)) 
p2 = multiprocessing.Process(target=api[0].download_all, args=(jan_data,))
p3 = multiprocessing.Process(target=api[0].download_all, args=(feb_data,))   
p4 = multiprocessing.Process(target=api[1].download_all, args=(mar_data,))      
# starting processes 
p1.start()
p2.start()
p3.start()
p4.start()
p1.join() 
p2.join()
p3.join()
p4.join()

In [ ]:
len(list(dec_data.keys()))

In [ ]:
od1 = OrderedDict() 
od2 = OrderedDict() 
od3 = OrderedDict() 
od4 = OrderedDict() 
od5 = OrderedDict() 
od6 = OrderedDict() 